In [10]:
class Person:
    """ 实例对象具有name属性，‘真实属性’为_name, 封装属性为name
    对name进行类型检查，设置属性必须为字符串；不可删除"""
    def __init__(self, value):
        self.name = value  # 触发name.setter(name)方法,即在初始化实例时即可进行类型检查
    
    @property
    def name(self):
        return self._name
    
    @name.setter
    def name(self, value):
        if isinstance(value, str):
            self._name = value
        else:
            raise TypeError('Expected a string')
    
    @name.deleter
    def name(self):
        raise AttributeError("Can't delete attribute")

In [3]:
p1 = Person('cxd')
p1.name

'cxd'

In [4]:
p1.name = 'cxy'
p1.name

'cxy'

In [5]:
p1.name = 18

TypeError: Expected a string

In [6]:
del p1.name

AttributeError: Can't delete attribute

In [7]:
p2 = Person(18)

TypeError: Expected a string

In [29]:
class SubPerson(Person):
    """ 扩展property
    父类.封装属性.魔术方法（__get__、 __set__、 __delete__）"""
    @property
    def name(self):
        print('Getting name')
        # 也可return super().name， return self.name 将导致无限“get”循环
        return super(SubPerson, SubPerson).name.__get__(self)
            
    @name.setter
    def name(self, value):
        super(SubPerson, SubPerson).name.__set__(self, value)    # super().name = value 错误
        print('Setting name to', value)

    @name.deleter
    def name(self):
        super(SubPerson, SubPerson).name.__delete__(self)  # 调用父类函数形式
        print('Deleting name')

In [30]:
sp = SubPerson('Guido')

Setting name to Guido


In [31]:
sp.name

Getting name


'Guido'

In [32]:
sp.name = 'Larry'

Setting name to Larry


In [33]:
sp.name = 42

TypeError: Expected a string

In [34]:
sp.__dict__

{'_name': 'Larry'}

In [22]:
import math

class LazyProperty:
    """延迟计算描述器类"""
    def __init__(self, func):
        self.func = func
    
    def __get__(self, instance, cls):
        """ instance是cls的实例，即将嵌入的类"""
        if instance is None: # 通过cls.描述器实例对象调用
            return self
        else:
            property_name = self.func.__name__
            if property_name not in instance.__dict__:
                print(f"Computing {property_name}")
                value = self.func(instance)
                instance.__dict__[property_name] = value
                return value
            else:
                return instance.__dict__[property_name]
    
    def __set__(self, instance, value):
        raise AttributeError("Can't set attribute, read only")
    
    def __delete__(self, instance):
        raise AttributeError("Can't delete attribute")
                
    

class Circle:
    """ 初始时进行类型检查[radius属性设置为只读]， 周长面积使用延迟计算"""
    def __init__(self, value):
        if isinstance(value, int) or isinstance(value, float):
            if value > 0:
                self._radius = value
            else:
                raise ValueError('expect positive radius')
        else:
            raise TypeError('expect numeric type')
    
    # 只读属性 radius
    @property
    def radius(self):
        return self._radius
    
    @radius.setter
    def radius(self, value):
        raise AttributeError("Can't set attribute, read only")
    
    @radius.deleter
    def radius(self):
        raise AttributeError("Can't delete attribute")
        
    # 延迟计算面积
    @LazyProperty
    def area(self):
        return math.pi * self.radius ** 2
    
    # 延迟计算周长
    @LazyProperty
    def perimeter(self):
        return 2 * math.pi * self.radius

In [15]:
c = Circle(4)
c.radius

4

In [16]:
c.radius = 5

AttributeError: Can't set attribute, read only

In [17]:
del c.radius

AttributeError: Can't delete attribute

In [18]:
c = Circle('cxd')

TypeError: expect numeric type

In [23]:
c = Circle(4.0)
c.radius

4.0

In [24]:
c.area

Computing area


50.26548245743669

In [25]:
c.area

50.26548245743669

In [26]:
c.perimeter

Computing perimeter


25.132741228718345

In [27]:
c.perimeter

25.132741228718345

In [28]:
c.area = 25

AttributeError: Can't set attribute, read only